In [6]:

# folder_path = "CSV_Output/"  # Update with your actual path
# all_features = []

import cupy as cp
import tensorflow as tf
import antropy as ant
import pandas as pd
import os
from scipy.stats import entropy

def hjorth_parameters(signal_gpu):
    first_derivative = cp.diff(signal_gpu)
    second_derivative = cp.diff(first_derivative)
    activity = cp.var(signal_gpu)
    mobility = cp.sqrt(cp.var(first_derivative) / activity)
    complexity = cp.sqrt(cp.var(second_derivative) / cp.var(first_derivative)) / mobility
    return activity, mobility, complexity

def feature_extraction(signal, file_name):
    signal_gpu = cp.asarray(signal)  # Move data to GPU
    
    mean_val = cp.mean(signal_gpu)
    std_val = cp.std(signal_gpu)
    rms_val = cp.sqrt(cp.mean(signal_gpu ** 2))
    
    kurtosis_val = cp.mean((signal_gpu - mean_val) ** 4) / (std_val ** 4)  # Kurtosis Approx.
    skewness_val = cp.mean((signal_gpu - mean_val) ** 3) / (std_val ** 3)  # Skewness Approx.
    
    # Shannon Entropy using SciPy with fixed bin size
    hist, _ = cp.histogram(signal_gpu, bins=50)  
    shannon_entropy_val = tf.constant(entropy(cp.asnumpy(hist)))
    
    # Convert signal back to TensorFlow tensor to compute entropy on GPU
    signal_tf = tf.convert_to_tensor(cp.asnumpy(signal_gpu), dtype=tf.float32)
    
    perm_entropy_val = tf.py_function(func=ant.perm_entropy, inp=[signal_tf, True], Tout=tf.float32)
    sample_entropy_val = tf.py_function(func=ant.sample_entropy, inp=[signal_tf], Tout=tf.float32)
    approx_entropy_val = tf.py_function(func=ant.app_entropy, inp=[signal_tf], Tout=tf.float32)
    
    activity, mobility, complexity = hjorth_parameters(signal_gpu)
    
    return {
        "File Name": file_name,
        "Mean": mean_val.get(),
        "Std Dev": std_val.get(),
        "Kurtosis": kurtosis_val.get(),
        "Skewness": skewness_val.get(),
        "Shannon Entropy": shannon_entropy_val.numpy(),
        "Activity": activity.get(),
        "Mobility": mobility.get(),
        "Complexity": complexity.get(),
        "Permutation Entropy": perm_entropy_val.numpy(),
        "Sample Entropy": sample_entropy_val.numpy(),
        "Approximate Entropy": approx_entropy_val.numpy()
    }

# Folder containing 79 CSV files
folder_path = "CSV_Output/"  # Update with your actual path
all_features = []

for file in os.listdir(folder_path):
    if file.endswith(".csv"):
        file_path = os.path.join(folder_path, file)
        signal = pd.read_csv(file_path).values.flatten()  # Read signal from CSV
        features = feature_extraction(signal, file)
        all_features.append(features)

# Save all extracted features to a single CSV file
df = pd.DataFrame(all_features)
df.to_csv("all_features.csv", index=False)
print("All features saved to all_features.csv")


InvalidArgumentError: {{function_node __wrapped__EagerPyFunc_Tin_2_Tout_1_device_/job:localhost/replica:0/task:0/device:CPU:0}} TypeError: Cannot convert 1 to EagerTensor of dtype bool
Traceback (most recent call last):

  File "c:\Users\Yash Srivastava\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow\python\ops\script_ops.py", line 269, in __call__
    return func(device, token, args)

  File "c:\Users\Yash Srivastava\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow\python\ops\script_ops.py", line 147, in __call__
    outputs = self._call(device, args)

  File "c:\Users\Yash Srivastava\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow\python\ops\script_ops.py", line 154, in _call
    ret = self._func(*args)

  File "c:\Users\Yash Srivastava\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow\python\autograph\impl\api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "c:\Users\Yash Srivastava\AppDa